This lab on Cross-Validation is a python adaptation of p. 190-194 of "Introduction to Statistical Learning
with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Written
by R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).

# 5.3.1 The Validation Set Approach

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In this section, we'll explore the use of the validation set approach in order to estimate the
test error rates that result from fitting various linear models on the ${\tt Auto}$ data set.

In [7]:
# df1 = pd.read_csv('data/Auto.csv', na_values='?').dropna()
df1 = pd.read_csv('data/Auto.csv')
df1.head(15)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl


We begin by using the ${\tt sample()}$ function to split the set of observations
into two halves, by selecting a random subset of 196 observations out of
the original 392 observations. We refer to these observations as the training
set.

We'll use the ${\tt random\_state}$ parameter in order to set a seed for
${\tt python}$’s random number generator, so that you'll obtain precisely the same results as those shown below. It is generally a good idea to set a random seed when performing an analysis such as cross-validation
that contains an element of randomness, so that the results obtained can be reproduced precisely at a later time.

In [9]:
train = df1.sample(196, random_state=1)
# test = df1[~df1.isin(train)].dropna(how = 'all')
test = df1[~df1.isin(train)]
type(train)

pandas.core.frame.DataFrame

We then use the ${\tt sm.OLS.from\_formula()}$ to fit a linear regression to predict ${\tt mpg}$ from ${\tt horsepower}$ using only
the observations corresponding to the training set.

In [4]:
lm = sm.OLS.from_formula('mpg~horsepower', train)
result = lm.fit()

We now use the ${\tt predict()}$ function to estimate the response for the test
observations, and we use some ${\tt numpy}$ functions to caclulate the MSE.

In [5]:
pred = result.predict(test)

MSE = np.mean(np.square(np.subtract(test["mpg"], pred)))
    
print(MSE)

23.36190289258723


**Note:** The MSE value here is different than that in the book because we're probably splitting the data in different test and train sets due to the likely different seeds for the random number generators.

In [6]:
mse2 = np.mean((test["mpg"] - pred)**2)
mse2

23.36190289258723

Therefore, the estimated test MSE for the linear regression fit is 23.36. We
can use the ${\tt np.power()}$ function to estimate the test error for the polynomial
and cubic regressions.

In [10]:
lm2 = sm.OLS.from_formula('mpg~' + '+'.join(['np.power(horsepower,' + str(i) + ')' for i in [1,2]]), train)
print(np.mean(np.square(np.subtract(test["mpg"], lm2.fit().predict(test)))))

lm3 = sm.OLS.from_formula('mpg~' + '+'.join(['np.power(horsepower,' + str(i) + ')' for i in [1,2,3]]), train)
print(np.mean(np.square(np.subtract(test["mpg"], lm3.fit().predict(test)))))

20.252690858350196
20.325609365878574


In [12]:
lm2_copy = sm.OLS.from_formula('mpg ~ horsepower + np.square(horsepower)', train)
print(np.mean(np.square(np.subtract(test["mpg"], lm2_copy.fit().predict(test)))))

20.252690858350196


These error rates are 20.25 and 20.33, respectively. If we choose a different
training set instead, then we will obtain somewhat different errors on the
validation set. We can test this out by setting a different random seed:

In [13]:
train = df1.sample(196, random_state = 2)

lm = sm.OLS.from_formula('mpg~horsepower', train)
print(np.mean(np.square(np.subtract(test["mpg"], lm.fit().predict(test)))))

lm2 = sm.OLS.from_formula('mpg~' + '+'.join(['np.power(horsepower,' + str(i) + ')' for i in [1,2]]), train)
print(np.mean(np.square(np.subtract(test["mpg"], lm2.fit().predict(test)))))

lm3 = sm.OLS.from_formula('mpg~' + '+'.join(['np.power(horsepower,' + str(i) + ')' for i in [1,2,3]]), train)
print(np.mean(np.square(np.subtract(test["mpg"], lm3.fit().predict(test)))))

23.214272449679584
19.525710963116943
19.66762809707753


Using this split of the observations into a training set and a validation
set, we find that the validation set error rates for the models with linear,
quadratic, and cubic terms are 23.21, 19.53, and 19.67, respectively.

These results are consistent with our previous findings: a model that
predicts ${\tt mpg}$ using a quadratic function of ${\tt horsepower}$ performs better than
a model that involves only a linear function of ${\tt horsepower}$, and there is
little evidence in favor of a model that uses a cubic function of ${\tt horsepower}$.

In [ ]:
from sklearn.cross_validation import LeaveOneOut

loo = LeaveOneOut(10)
for train_index, test_index in loo:
    df1[train_index]